# Rainfall Prediction in Australia

## Problem Statement:

Time series analysis over the weather data of Australia. And Predictive modeling to answer wheather tomorrow is going to rain or not


# Objective:

Give me Chance to learn from your importent feedback.If you like this please don't forget to leave a comment. Advance  thanks for your time.
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV


In [ ]:
df = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().mean()*100

Here we are seeing that, there are four features which contains around (45-50)% of missing value. And we cann't also just delete them, because those could greatly impact in our modeling.So Our first task is to check this then start proceding.



In [ ]:
Miss_tar = df[['Evaporation', 'Sunshine', 'Cloud9am','Cloud3pm','RainTomorrow']]

In [ ]:
Miss_tar['RainTomorrow'] = Miss_tar['RainTomorrow'].fillna('No')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Miss_tar['RainTomorrow'] = le.fit_transform(Miss_tar['RainTomorrow'])

In [ ]:
sns.heatmap(Miss_tar.corr(), cmap = 'RdBu', annot = True, linewidths=1, linecolor='black')

Here we are seeing that Cloud9am and Cloud3pm are roughly releated to our target variable and there is a good relationship between them also.And rest two are badly related with target and between themselves also. So let's go with this two and let's see...

In [ ]:
df = df.drop(['Evaporation','Sunshine'], axis = 1)
df['Cloud9am'] = df.Cloud9am.mean()
df['Cloud3pm'] = df.Cloud3pm.mean()
df.dropna(inplace = True)
np.round(df.isnull().mean())

In [ ]:
df.columns

#Let's have look at our variable

Variable Types

1. Numerical variables:
   'MinTemp', 'MaxTemp', 'Rainfall','WindGustSpeed','WindSpeed9am',
   'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am',
   'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm',

2. Categorical variables:
   'Date', 'Location','WindGustDir','WindDir9am','WindDir3pm','RainToday', 'RainTomorrow'


In [ ]:
# find numerical variables

numerical = [var for var in df.columns if df[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :', numerical)

df_num = df[numerical]

In [ ]:
categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} numerical variables\n'.format(len(categorical)))

print('The numerical variables are :', categorical)
df_cat = df[categorical]

Let's Analyse the data :

Let's start with spliting the date colimn into year, month and day.

In [ ]:
#first convert this 'date' column into datetime type
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['Month'] = df['Date'].dt.month
df['Weakday'] = df['Date'].dt.weekday
df['Day'] = df['Date'].dt.day


In [ ]:
#Setting Date column as index
df = df.set_index('Date')

Before dig deep into time series analysis let's see how rain occurs with respect to Years.



In [ ]:
sns.set_style('darkgrid')
#sns.set_palette('Set1')
plt.figure(figsize=[15,6])
ax = sns.countplot(x = 'Year', hue = 'RainTomorrow',edgecolor=(0,0,0),
                  linewidth=2,
              palette="Accent_r", data = df)
ax.set_title( "Occurence of rain in Years",size = 30 )
plt.show()

Here we are seeing a seasonality.From a Certain year Rain starts increasing and after 3-4 years it's reaches to pick and start decreasing.........

In [ ]:
sns.set_style('darkgrid')
sns.set_palette('Set1')
plt.figure(figsize=[15,6])
ax = sns.countplot(x = 'Month', hue = 'RainTomorrow',edgecolor=(0,0,0),
                  linewidth=2,
              data = df)
ax.set_title( "Occurence of rain in Month",size = 30 )
plt.show()

Here we are seeing that middle of the year rain occurs most frequently, specially in May, June, July, August.



In [ ]:
sns.set_style('darkgrid')
sns.set_palette('Set2')
plt.figure(figsize=[15,6])
ax = sns.scatterplot(x = 'Month', y = 'Rainfall',data = df)
ax.set_title( "Amount of  rain in Month",size = 30 )
plt.show()


Here some interesting things to notice.

Though rain is more frequent in the middle of the year, But the amount of rainfall is much more in Jan, Feb and later half of the year.

In many countries rain occurs in particular period of a year, but in australia rain is available in whole through the years.



In [ ]:
sns.set_style('darkgrid')
sns.set_palette('spring_r')
plt.figure(figsize=[15,6])
ax = sns.countplot(x = 'Day', hue = 'RainTomorrow',edgecolor=(0,0,0),
                  linewidth=2,
              data = df)
ax.set_title( "Occurence of rain in day of month",size = 30 )
plt.show()


There are no such difference. Possibility of rain in the day range between 3-4, 8-11, 21-24 is slightly higher then others.



In [ ]:
sns.set_style('darkgrid')
sns.set_palette('plasma')
plt.figure(figsize=[15,6])
ax = sns.scatterplot(x = 'Day', y = 'Rainfall',data = df)
ax.set_title( "Amount of rain in day of month",size = 30 )
plt.show()

Here We are seeing that starting couple of days and day range 12-14 and 20-27 having a havier rainfall.



In [ ]:
sns.set_style('darkgrid')
sns.set_palette('plasma_r')
plt.figure(figsize=[15,6])
ax = sns.countplot(x = 'Quarter', hue = 'RainTomorrow',edgecolor=(0,.78,1),
                  linewidth=3,
              data = df)
ax.set_title( "Occurence of rain in Quarter",size = 30 )
plt.show()


In [ ]:
sns.set_style('darkgrid')
sns.set_palette('plasma_r')
plt.figure(figsize=[15,6])
ax = sns.scatterplot(x = 'Quarter', y = 'Rainfall',data = df)
ax.set_title( "Amount of rain in Quarter",size = 30 )
plt.show()


# Rules of Temperature:
If Temperature increases Pressure of air will be decreassed.

In [ ]:
#Let's make new column to check how temp is changing.
df['Temp_diff'] = df['Temp3pm'] - df['Temp9am']

In [ ]:
fig = plt.figure()
x = (df[['MaxTemp','Temp_diff','Rainfall','WindGustSpeed']])
x.plot(grid =True, subplots = True,figsize = (50,30), fontsize = 50, legend = 'reverse')
#title = 'How Numerical value changes with respect to time '
#fig.suptitle = 'Temparature vs Rainfall Vs WindGustSpeed'
plt.grid()
plt.show()

Observations: From the above chart we are seeing that.

1.At the starting and ending of the year varies from 20-45 and in the middle 10-30.

2.Temperature start to decreases in the middle of the year and then again increasing as starting of the year.

3.Almost 90% difference of temparature between 9am and 3 Pm are positive.Whice means at the later halfof the dat temparature start increasing. That's why we are expecting of heavy rainfall at the end of the day and more frequent rain at the starting of the day.

3.There is a relationship between Maxtemp and WindGustspeed. in the middle of the year WindGustspeed Start decresing.Which means The higher temperature is causing for havier WindGustspeed.

As a result Australia has heavy rainfall during starting and ending of the year and more frequent in middle of the year.



In [ ]:
df[['Month', 'MaxTemp', 'Rainfall','WindGustSpeed']].groupby('Month').agg({'MaxTemp':['mean','max'], 'Rainfall':['min','max'], 'WindGustSpeed':['mean','max']})

What we have seen on the graphs are prejected in the above numeical value.

# WindSpeed:

In [ ]:
df['WindSpeed_diff'] = df['WindSpeed3pm'] - df['WindSpeed9am']
#plt.figure(figsize=[50,300])
x = (df[['MaxTemp','WindSpeed_diff','WindSpeed9am','WindSpeed3pm']])
x.plot(grid =True, subplots = True,figsize = (50,30), fontsize = 50) 
#plt.title = 'How Numerical value changes with respect to time '

plt.grid()
plt.show()

Observations:

1.There is no relation between Temp and wind speed.

2.At the end of the day the lower value od wind speed slightly increased.

3.And difference is 50-50 positive and negetive.

So don't think windspeed_diff will be handy and delete it.



In [ ]:
df.drop(['WindSpeed_diff'], axis=1,inplace =True)

# Pressure:

In [ ]:
df['WindPressure_diff'] = df['Pressure3pm'] - df['Pressure9am']
#plt.figure(figsize=[50,300])
x = (df[['MaxTemp','WindPressure_diff','Pressure9am','Pressure3pm']])
x.plot(grid =True, subplots = True,figsize = (50,30), fontsize = 50) 
#plt.title = 'How Numerical value changes with respect to time '

plt.grid()
plt.show()

Observations:

I think it need not to be explained.Because it's totally resembling to the fact of law of Temparature.



In [ ]:
df[['Month', 'MaxTemp', 'Pressure9am','Pressure3pm']].groupby('Month').agg({'MaxTemp':['mean','max'], 'Pressure9am':['mean','max'], 'Pressure3pm':['mean','max']})

# Cloud:

In [ ]:
df['Cloud_diff'] = df['Cloud3pm'] - df['Cloud9am']
#plt.figure(figsize=[50,300])
x = (df[['Cloud_diff','Cloud9am','Cloud3pm']])
x.plot(grid =True, subplots = True,figsize = (50,30), fontsize = 50) 
plt.title = 'How Numerical value changes with respect to time '

plt.grid()
plt.show()

Observations:

Here we are seeing that The amount of sky covered by cloud is same through out year.At the end of the day it slightly increased but remain same.But rain changes throughout the year. So we can say that it has almost no effect in rain.



# Humidity:

In [ ]:
#plt.figure(figsize=[50,300])
df['Humidity_diff'] = df['Humidity3pm'] - df['Humidity9am']
x = (df[['Humidity_diff','Humidity9am','Humidity3pm']])
x.plot(grid =True, subplots = True,figsize = (60,30), fontsize = 50) 
#plt.title = 'How Numerical value changes with respect to time '

plt.grid()
plt.show()

Obsevations:

It's slightly simmiliar to the Temperature and at the middle of the year humidity decreases little bit.



In [ ]:
df[['Month', 'Humidity9am','Humidity3pm']].groupby('Month').agg({ 'Humidity9am':['mean','max'], 'Humidity3pm':['mean','max']})

It's time to explore the categorical variable.

# Location

In [ ]:
print('Location contains', len(df['Location'].unique()), 'labels')

In [ ]:
df.Location.unique()

In [ ]:
df.Location.value_counts()

# WindGustDir:

In [ ]:
print('WindGustDir contains', len(df['WindGustDir'].unique()), 'labels')

In [ ]:
plt.figure(figsize = (15,5))
sns.countplot(x = 'WindGustDir', hue = 'RainTomorrow', data = df,
             edgecolor=(0,.78,1), linewidth = 2)

# WindDir9am:

In [ ]:
print('WindDir9am contains', len(df['WindDir9am'].unique()), 'labels')

In [ ]:
plt.figure(figsize = (15,5))
sns.countplot(x = 'WindDir9am', hue = 'RainTomorrow', data = df,
             edgecolor=(.23,.78,.5), linewidth = 2)

# WindDir3pm:

In [ ]:
plt.figure(figsize = (15,5))
sns.countplot(x = 'WindDir3pm', hue = 'RainTomorrow', data = df,
             edgecolor=(.23,.1,.5), linewidth = 2)

# RainToday:

In [ ]:
plt.figure(figsize = (8,4))
sns.countplot(x = 'RainToday', hue = 'RainTomorrow', data = df,
             edgecolor=(.23,.78,.5), linewidth = 2, palette="prism")

Here a importent point to notice is One can say with 95% confident that tomorrow is going to rain, if today rain does occurs.

It's time to see the correlation among the features.

We are expecting of relationship between all the simillar types of columns like, maxTemp, minTemp, Pressure 9am and 3pm, Humidity at 9 am and 3 Pm.....

There can be a positive relationship between Temp and WindGustspeed, and Temp and Humidity.

And a negetive relationship between Temp and Pressure.

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(df.corr(), cmap = 'RdBu', annot = True, linewidths=1, linecolor='black')

My assumptions are come to truth.

Column that are positively related with Temperature are negetively related with Pressure.

And there is a strong relation between features of same types.



# Outlier removal:


In [ ]:
print(round(df[numerical].describe()))

On closer inspection, we can see that the Rainfall,WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity3pm columns may contain outliers.

I will draw boxplots to visualise outliers in the above variables.



In [ ]:
# draw boxplots to visualize outliers

plt.figure(figsize=(15,10))

plt.subplot(3, 2, 1)
fig = df.boxplot(column='Rainfall')
fig.set_title('')
fig.set_ylabel('Rainfall')

plt.subplot(3, 2, 2)
fig = df.boxplot(column='WindGustSpeed')
fig.set_title('')
fig.set_ylabel('WindGustSpeed')

plt.subplot(3, 2, 3)
fig = df.boxplot(column='WindSpeed9am')
fig.set_title('')
fig.set_ylabel('WindSpeed9am')

plt.subplot(3, 2, 4)
fig = df.boxplot(column='WindSpeed3pm')
fig.set_title('')
fig.set_ylabel('WindSpeed3pm')

plt.subplot(3, 2, 5)
fig = df.boxplot(column='Humidity3pm')
fig.set_title('')
fig.set_ylabel('Humidity3pm')

In [ ]:
column = ['Rainfall','WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity3pm']

In [ ]:
def Outlier_detection(df, column):
    for i in column:
        IQR = df[i].quantile(0.75) - df[i].quantile(0.25)
        l_b = df[i].quantile(0.25) - (IQR * 3)
        u_b = df[i].quantile(0.75) + (IQR * 3)
        
        med = np.median(df[i])
        
        df[i] = np.where(df[i] > u_b , med,
                         np.where(df[i] < l_b, med, df[i]))
Outlier_detection(df, column)


In [ ]:
df.describe()

# Encoding:

In [ ]:
df.columns

Here WeakDay, cloud_diff columns are notnecessary. So let's delete those.



In [ ]:
df.drop(['Weakday', 'Cloud_diff'], axis = 1, inplace = True)
df.head()

In [ ]:
le = LabelEncoder()
df['RainTomorrow'] = le.fit_transform(df['RainTomorrow'])

In [ ]:
df = pd.get_dummies(df,
                    columns=['Location','WindGustDir','WindDir9am','WindDir3pm','RainToday'],

                    drop_first=True)

Let's make the train test split

In [ ]:
X = df.drop(['RainTomorrow'], axis = 1)
y = df['RainTomorrow']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=42)

# Modeling

In [ ]:
model = []
precision = []
recall = []
F1score = []
Accuracy = []
AUCROC = []

# LogisticRegression:

In [ ]:
Log = LogisticRegression(C  = 10)

In [ ]:
Log.fit(X_train, y_train)

y_prob=Log.predict_proba(X_test)[:,1]
y_pred=Log.predict(X_test)

fpr, tpr, thresholds  = roc_curve(y_test, y_prob)

plt.rcParams['font.size'] = 12
plt.plot(fpr, tpr, color = 'blue', marker = 'o', markerfacecolor = 'red', markersize = 1)
plt.plot([0,1], [0,1], 'k--' )
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
#plt.title('ROC Curve for rain prediction CLassifier')
plt.xlabel('False Positive Rate(1-Specificity)')
plt.ylabel('True Positive Rate(Sensitivity)')
plt.grid(True)
plt.show()

print('Confusion Matrix')
print('='*60)
print(confusion_matrix(y_test,y_pred),"\n")
print('Classification Report')
print('='*60)
print(classification_report(y_test,y_pred),"\n")
print('AUC-ROC')
print('='*60)
print(roc_auc_score(y_test, y_prob))
print('Accuracy Score')
print('='*60)
print(accuracy_score(y_test, y_pred))
          
model.append('Logistic REgression')
precision.append(precision_score(y_test,y_pred))
recall.append(recall_score(y_test,y_pred))
F1score.append(f1_score(y_test,y_pred))
Accuracy.append(accuracy_score(y_test, y_pred))
AUCROC.append(roc_auc_score(y_test, y_prob))


# Naive Bayes CLassifier:

## GaussianNB:

In [ ]:
from sklearn.naive_bayes import GaussianNB
Naive_bayes= GaussianNB()

In [ ]:
Naive_bayes.fit(X_train, y_train)

y_prob=Naive_bayes.predict_proba(X_test)[:,1]
y_pred=Naive_bayes.predict(X_test)

fpr, tpr, thresholds  = roc_curve(y_test, y_prob)

plt.rcParams['font.size'] = 12
plt.plot(fpr, tpr, color = 'blue', marker = 'o', markerfacecolor = 'red', markersize = 1)
plt.plot([0,1], [0,1], 'k--' )
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
#plt.title('ROC Curve for rain prediction CLassifier')
plt.xlabel('False Positive Rate(1-Specificity)')
plt.ylabel('True Positive Rate(Sensitivity)')
plt.grid(True)
plt.show()

print('Confusion Matrix')
print('='*60)
print(confusion_matrix(y_test,y_pred),"\n")
print('Classification Report')
print('='*60)
print(classification_report(y_test,y_pred),"\n")
print('AUC-ROC')
print('='*60)
print(roc_auc_score(y_test, y_prob))
print('Accuracy Score')
print('='*60)
print(accuracy_score(y_test, y_pred))
          
model.append('GaussianNB')
precision.append(precision_score(y_test,y_pred))
recall.append(recall_score(y_test,y_pred))
F1score.append(f1_score(y_test,y_pred))
Accuracy.append(accuracy_score(y_test, y_pred))
AUCROC.append(roc_auc_score(y_test, y_prob))


## BernoulliNB

Work in Progress

In [ ]:
from sklearn.naive_bayes import BernoulliNB
Bernoulli= BernoulliNB()

In [ ]:
Bernoulli.fit(X_train, y_train)

y_prob=Bernoulli.predict_proba(X_test)[:,1]
y_pred=Bernoulli.predict(X_test)

fpr, tpr, thresholds  = roc_curve(y_test, y_prob)

plt.rcParams['font.size'] = 12
plt.plot(fpr, tpr, color = 'blue', marker = 'o', markerfacecolor = 'red', markersize = 1)
plt.plot([0,1], [0,1], 'k--' )
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
#plt.title('ROC Curve for rain prediction CLassifier')
plt.xlabel('False Positive Rate(1-Specificity)')
plt.ylabel('True Positive Rate(Sensitivity)')
plt.grid(True)
plt.show()

print('Confusion Matrix')
print('='*60)
print(confusion_matrix(y_test,y_pred),"\n")
print('Classification Report')
print('='*60)
print(classification_report(y_test,y_pred),"\n")
print('AUC-ROC')
print('='*60)
print(roc_auc_score(y_test, y_prob))
print('Accuracy Score')
print('='*60)
print(accuracy_score(y_test, y_pred))
          
model.append('BernoulliNB')
precision.append(precision_score(y_test,y_pred))
recall.append(recall_score(y_test,y_pred))
F1score.append(f1_score(y_test,y_pred))
Accuracy.append(accuracy_score(y_test, y_pred))
AUCROC.append(roc_auc_score(y_test, y_prob))


Work in Progress